<a href="https://colab.research.google.com/github/suyogpradhan0/Spatial-Data-Management-with-Google-Earth-Engine/blob/main/Visualize_NOAA_GFS_Temperature_Data%2C_2_2_linked_map_of_Landsat_imagery%2C_Timeseries_inspector_of_USDA_NASS_Cropland_Data_Layers%2C_Time_slider_of_Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 6

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_06.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Question 1

Create a map to visualize [NOAA GFS Temperature Data](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25) and add a color bar and [NOAA logo](https://i.imgur.com/spILFEi.png) to the map.

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

In [ ]:
# Load NOAA GFS temperature data for the specified date range and region
collection = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2018-12-22', '2018-12-23')
    .limit(24)
    .select('temperature_2m_above_ground')
)

# Create a map
Map = geemap.Map()

# Add the temperature data to the map with a custom color palette
vis_params = {
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
Map.addLayer(collection, vis_params, 'NOAA GFS Temperature')

# Add a color bar to the map
Map.add_colorbar(vis_params, label="Temperature Above Ground", orientation="horizontal", layer_name="NOAA GFS Temperature")

# Add the NOAA logo to the map
noaa_logo = 'https://www.space.commerce.gov/wp-content/uploads/logo-noaa.png'
Map.add_image(noaa_logo, position='bottomleft')

#Add text to the map
text = "Made by Suyog Pradhan"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/bBoRswR.png)

## Question 2

**Linked Maps:** Create a 2*2 linked map to visualize the Landsat imagery (`ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`) with different band combinations.

In [ ]:
# Load a Landsat 7 ImageCollection for the time period 1999-2003
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

# Define visualization parameters for different band combinations
vis_params = [
    {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 255, 'gamma': 1.4},
    {'bands': ['B4', 'B5', 'B3'], 'min': 0, 'max': 255, 'gamma': 1.4},
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 255, 'gamma': 1.4},
    {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 255, 'gamma': 1.4},
]

# Define labels for each band combination
labels = [
    'Natural Color (B3/B2/B1)',
    'Land/Water (B4/B5/B3)',
    'Color Infrared (B4/B3/B2)',
    'Vegetation (B5/B4/B3)',
]

# Display linked maps in a 2x2 grid
geemap.linked_maps(
    rows=2,
    cols=2,
    height="300px",
    center=[38.4151, 21.2712],  # Set the center of the maps
    zoom=12,  # Set the zoom level of the maps
    ee_objects=[image],  # Pass the image to be displayed in each map
    vis_params=vis_params,  # Pass the visualization parameters for each map
    labels=labels,  # Pass the labels for each map
    label_position="topright",  # Set the position of the labels
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

![](https://i.imgur.com/Xp15uOo.png)

## Question 3

**Timeseries Inspector:** Create a map with timeseries inspector to visualize [USDA NASS Cropland Data Layers](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL) from 2010 to 2022. Add your name and [USDA logo](https://i.imgur.com/tzH2dNr.png) to the map.

In [ ]:
# Create a map centered at USA
Map = geemap.Map(center=[40, -100], zoom=4)

# Define the start and end years for filtering the USDA NASS Cropland Data Layers
start_year = 2010
end_year = 2022

# Load the USDA NASS Cropland Data Layers, select the 'cropland' band, and filter by date
collection = ee.ImageCollection('USDA/NASS/CDL').select('cropland').filterDate(f'{start_year}-01-01', f'{end_year}-12-31')

# Define visualization parameters for the 'cropland' band
vis_params = {'bands': ['cropland']}

# Get the unique years present in the collection
years = collection.aggregate_array('system:index').getInfo()

# Add a time series inspector to the map
Map.ts_inspector(
    left_ts=collection,
    right_ts=collection,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',  # Set the width of the time series inspector
)

# Add the USDA logo and name to the map
usda_logo = 'https://seeklogo.com/images/U/USDA-logo-81BBA5ED63-seeklogo.com.png'
Map.add_image(usda_logo, position='bottomleft')
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=…

![](https://i.imgur.com/gW1XPhL.png)

## Question 4

**Time slider:** Create a map with the time slider to visualize [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) for Knoxville, TN.

In [ ]:
# Define the region of interest (Knoxville, TN)
roi = ee.Geometry.Point([-83.9466663, 35.966096])

# Create a Sentinel-2 image collection for the specified area and time period
collection = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .filterDate('2018-01-01', '2018-12-31')
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
              .select(['B11', 'B8', 'B2']))

# Create a map centered on Knoxville, TN
Map = geemap.Map(center=[35.966096, -83.9466663], zoom=13)

# Add the Sentinel-2 image collection to the map with a time slider
Map.add_time_slider(collection, {'bands': ['B11', 'B8', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}, time_interval=2)

# Display the map with the time slider
Map.add_text(text, fontsize=20, position='bottomright')
Map

Map(center=[35.966096, -83.9466663], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

![](https://i.imgur.com/rJe2rvP.png)

## Question 5

**Split-panel Map:** Use the following datasets to create a split-panel map for visualizing the ESA land cover data in the US. Add the ESA land cover legend to the map (Hint: the built-in legend for ESA land cover is `ESA_WorldCover`).

- [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States): `ee.FeatureCollection("TIGER/2018/States")`
- [ESA WorldCover 10m](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100): `ee.ImageCollection("ESA/WorldCover/v100")`
- Landsat: `LANDSAT/LE7_TOA_5YEAR/1999_2003`


Currently, the split-map control of ipyleaflet plotting backend has a bug ([source](https://github.com/jupyter-widgets/ipyleaflet/issues/1066)). Use the folium plotting backend instead.

In [ ]:
# Load US Census States
states = ee.FeatureCollection("TIGER/2018/States")

# Load ESA WorldCover 10m
esa_worldcover = ee.ImageCollection("ESA/WorldCover/v100").first()

# Load Landsat imagery for the time period 1999-2003
landsat_image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

# Add the Landsat imagery to the left map
left_layer = geemap.ee_tile_layer(landsat_image.clip(states), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 255, 'gamma': 1.4}, 'Landsat Image')

# Add the ESA WorldCover to the right map
right_layer = geemap.ee_tile_layer(esa_worldcover.clip(states), {}, 'ESA WorldCover')

# Create a map centered on the continental United States
Map = geemap.Map(center=[40, -100], zoom=4)
Map.split_map(left_layer, right_layer, add_close_button=True)

# Add the ESA land cover legend to the map
ee_class_table = """
Value	Color	Description
10	006400	Trees
20	ffbb22	Shrubland
30	ffff4c	Grassland
40	f096ff	Cropland
50	fa0000	Built-up
60	b4b4b4	Barren / sparse vegetation
70	f0f0f0	Snow and ice
80	0064c8	Open water
90	0096a0	Herbaceous wetland
95	00cf75	Mangroves
100	fae6a0	Moss and lichen
"""

legend_dict = geemap.legend_from_ee(ee_class_table)

Map.add_legend(title='ESA Land Cover', legend_dict=legend_dict, colors=None)

# Display the split-panel map
Map.add_text(text, fontsize=20, position='bottomright')
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

![](https://i.imgur.com/y0GW6k8.png)